## Country-level INFRA-SAP

This notebook exemplifies a simple market access estimation based on global datasets:

- **Global Friction Surface (Malaria Atlas Project)**  
see: https://developers.google.com/earth-engine/datasets/catalog/Oxford_MAP_friction_surface_2019


- **World Pop 1 km Population Grid**

In [1]:
import os, sys, time, importlib

import geopandas as gpd
import pandas as pd
import networkx as nx
sys.path.append('/home/wb514197/Repos/GOSTnets')

import GOSTnets as gn
import GOSTnets.calculate_od_raw as calcOD
from GOSTnets.load_osm import *
import rasterio as rio
from osgeo import gdal
import numpy as np
from shapely.geometry import Point

sys.path.append('/home/wb514197/Repos/INFRA_SAP')
from infrasap import aggregator
from utm_zone import epsg as epsg_get
import json

%load_ext autoreload
%autoreload 2

In [2]:
sys.path.append('/home/wb514197/Repos/gostrocks/src')
sys.path.append('/home/wb514197/Repos/GOSTNets_Raster/src')

In [3]:
import GOSTRocks.rasterMisc as rMisc
import GOSTNetsRaster.market_access as ma
import skimage.graph as graph

### Load origins and graph

In [5]:
# base_in = "/home/public/Data/PROJECTS/INFRA_SAP"
base_in = "/home/wb514197/data/INFRA_SAP"
in_folder = os.path.join(base_in, iso3)

# define data paths
focal_admin2 = os.path.join(in_folder, "admin.shp")
focal_osm = os.path.join(in_folder, f"{country}-latest.osm.pbf")
pop_name = "WP_2020_1km"
wp_1km = os.path.join(in_folder, f"{pop_name}.tif")
urban_extents = os.path.join(in_folder, "urban_extents.shp")
airports = os.path.join(in_folder, "airports.shp")
ports = os.path.join(in_folder, "ports.shp")
borders = os.path.join(in_folder, "borders.shp")

out_folder = os.path.join(in_folder, "output")
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

### Clip friction surface and population to country extent

In [4]:
global_friction_surface = "/home/public/Data/GLOBAL/INFRA/FRICTION_2020/2020_motorized_friction_surface.geotiff"
global_population = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/ppp_2020_1km_Aggregated.tif"
inG = rio.open(global_friction_surface)
inP = rio.open(global_population)

# Read in country bounds
global_bounds = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
admin1 = "/home/public/Data/GLOBAL/ADMIN/Admin1_Polys.shp"
admin2 = "/home/wb514197/data/PAK/pakistan_indicators.shp"

inB = gpd.read_file(global_bounds)
inB = inB.loc[inB['ISO3'] == iso3]
inB = inB.to_crs(inG.crs.to_string())

In [5]:
# Clip the travel raster to ISO3
out_travel_surface = os.path.join(data_dir, "TRAVEL_SURFACE.tif")
rMisc.clipRaster(inG, inB, out_travel_surface)

# Clip the population raster to ISO3
out_pop_surface = os.path.join(data_dir, "POP_2020_NEW.tif")
rMisc.clipRaster(inP, inB, out_pop_surface)

In [8]:
travel_surf = rio.open(out_travel_surface)
pop_surf = rio.open(out_pop_surface)

In [7]:
# Make sure that both rasters have the exact same resolution, crs, and number of pixels
out_pop_surface_std = os.path.join(data_dir, "POP_2020_NEW_STD.tif")
rMisc.standardizeInputRasters(pop_surf, travel_surf, os.path.join(data_dir, "POP_2020_NEW_STD.tif"), data_type="C")

### Prepare Raster

In [15]:
travel_surf = rio.open(out_travel_surface)

In [11]:
pop_surf = rio.open(out_pop_surface_std)
pop = pop_surf.read(1, masked=True)
indices = list(np.ndindex(pop.shape))
xys = [pop_surf.xy(ind[0], ind[1]) for ind in indices]
res_df = pd.DataFrame({
    'spatial_index': indices, 
    'xy': xys, 
    'pop': pop.flatten()
})
res_df['pointid'] = res_df.index

In [12]:
len(res_df)

2510475

In [13]:
# create MCP object
inG_data = travel_surf.read(1) * 1000 #
# Correct no data values
inG_data[inG_data < 0] = 99999999
# inG_data[inG_data < 0] = np.nan
mcp = graph.MCP_Geometric(inG_data)

### Prepare destinations

In [14]:
inCities = gpd.read_file(urban_extents)
pop_thresh=50000
inCities = inCities.loc[inCities.Pop>pop_thresh]

In [15]:
inCities['geometry'] = inCities.geometry.representative_point()
dest_cities = inCities.assign(dest_type = 'city')

In [16]:
dest_airports = gpd.read_file(airports)
dest_airports = dest_airports.assign(dest_type = 'airport')

In [17]:
dest_ports = gpd.read_file(ports).assign(dest_type = 'port')
dest_borders = gpd.read_file(borders).assign(dest_type = 'border')
dest_borders['geometry'] = dest_borders.geometry.apply(lambda x: x[0])

In [18]:
dest_all = pd.concat([dest_cities, dest_airports, dest_ports, dest_borders], ignore_index=True) # dest_borders

In [19]:
dest_all.dest_type.value_counts()

border     32
city       29
airport    12
port        4
Name: dest_type, dtype: int64

In [21]:
dest_all.to_file(os.path.join(out_folder, 'destination_all.shp'), driver="ESRI Shapefile")

In [23]:
# for each destination get cost of travel for every origin
for idx, dest in tqdm(dest_all.iterrows()):
    dest_gdf = gpd.GeoDataFrame([dest], geometry='geometry', crs='EPSG:4326')
    res = ma.calculate_travel_time(travel_surf, mcp, dest_gdf)[0]
    res_df.loc[:,idx] = res.flatten()

77it [02:33,  1.99s/it]


In [24]:
res_df.head()

,spatial_index,xy,pop,pointid,0,1,2,3,4,5,...,67,68,69,70,71,72,73,74,75,76
0,"(0, 0)","(30.220833333333335, -10.470833333333335)",NaN,0,5.000203e+07,5.000183e+07,5.000134e+07,5.000101e+07,5.000196e+07,5.000141e+07,...,5.000118e+07,5.000118e+07,5.000117e+07,5.000117e+07,5.000122e+07,5.000122e+07,5.000129e+07,5.000129e+07,5.000221e+07,5.000221e+07
1,"(0, 1)","(30.229166666666668, -10.470833333333335)",NaN,1,5.000202e+07,5.000182e+07,5.000133e+07,5.000100e+07,5.000194e+07,5.000140e+07,...,5.000117e+07,5.000117e+07,5.000116e+07,5.000116e+07,5.000121e+07,5.000121e+07,5.000128e+07,5.000128e+07,5.000219e+07,5.000219e+07
2,"(0, 2)","(30.2375, -10.470833333333335)",NaN,2,5.000200e+07,5.000181e+07,5.000131e+07,5.000098e+07,5.000193e+07,5.000138e+07,...,5.000115e+07,5.000115e+07,5.000114e+07,5.000114e+07,5.000119e+07,5.000119e+07,5.000126e+07,5.000126e+07,5.000218e+07,5.000218e+07
3,"(0, 3)","(30.245833333333334, -10.470833333333335)",NaN,3,5.000201e+07,5.000181e+07,5.000131e+07,5.000099e+07,5.000193e+07,5.000138e+07,...,5.000116e+07,5.000116e+07,5.000114e+07,5.000114e+07,5.000119e+07,5.000119e+07,5.000127e+07,5.000127e+07,5.000218e+07,5.000218e+07
4,"(0, 4)","(30.25416666666667, -10.470833333333335)",NaN,4,5.000202e+07,5.000182e+07,5.000133e+07,5.000100e+07,5.000195e+07,5.000140e+07,...,5.000117e+07,5.000117e+07,5.000116e+07,5.000116e+07,5.000121e+07,5.000121e+07,5.000128e+07,5.000128e+07,5.000219e+07,5.000219e+07


In [25]:
# remove values where pop is 0 or nan
res_df = res_df.loc[res_df['pop']!=0].copy()
res_df = res_df.loc[~(res_df['pop'].isna())].copy()

In [26]:
res_df.loc[:,'xy'] = res_df.loc[:,'xy'].apply(lambda x: Point(x))

In [27]:
len(res_df)

959933

Remove values where travel time was undefined

In [28]:
res_df = res_df.loc[res_df[0]<99999999]

In [29]:
origins_join = res_df.copy()

In [30]:
all(origins_join.columns[4:] == dest_all.index)

True

In [31]:
res_df.columns

Index(['spatial_index',            'xy',           'pop',       'pointid',
                     0,               1,               2,               3,
                     4,               5,               6,               7,
                     8,               9,              10,              11,
                    12,              13,              14,              15,
                    16,              17,              18,              19,
                    20,              21,              22,              23,
                    24,              25,              26,              27,
                    28,              29,              30,              31,
                    32,              33,              34,              35,
                    36,              37,              38,              39,
                    40,              41,              42,              43,
                    44,              45,              46,              47,
                    48,  

In [32]:
origins_join_rename = origins_join.copy()
origins_join_rename.columns = pd.MultiIndex.from_arrays([['origin' for each in origins_join.columns[:4]]+list(dest_all.dest_type), origins_join.columns])

In [33]:
origins_join_rename.head()

origin                                                           \
     spatial_index                                            xy        pop   
1226     (0, 1226)            POINT (40.4375 -10.47083333333333)  12.027128   
1227     (0, 1227)  POINT (40.44583333333333 -10.47083333333333)  10.314653   
1228     (0, 1228)  POINT (40.45416666666667 -10.47083333333333)   8.879251   
1229     (0, 1229)  POINT (40.46250000000001 -10.47083333333333)   7.824205   
2500     (1, 1225)  POINT (40.42916666666667 -10.47916666666667)  11.924029   

                      city                                            \
     pointid             0             1             2             3   
1226    1226  5.000052e+07  5.000054e+07  5.000111e+07  5.000145e+07   
1227    1227  5.000055e+07  5.000057e+07  5.000114e+07  5.000148e+07   
1228    1228  5.000054e+07  5.000056e+07  5.000114e+07  5.000148e+07   
1229    1229  5.000054e+07  5.000056e+07  5.000113e+07  5.000147e+07   
2500    2500  5.145629e+02  5.352954e+02  1.110124e+03  1.448182e+03   

                                  ...        border                \
                 4             5  ...            67            68   
1226  5.000076e+07  5.000106e+07  ...  5.000147e+07  5.000147e+07   
1227  5.000079e+07  5.000109e+07  ...  5.000150e+07  5.000150e+07   
1228  5.000079e+07  5.000109e+07  ...  5.000149e+07  5.000149e+07   
1229  5.000078e+07  5.000108e+07  ...  5.000149e+07  5.000149e+07   
2500  7.586303e+02  1.059558e+03  ...  1.465990e+03  1.465990e+03   

                                                                            \
                69            70            71            72            73   
1226  5.000146e+07  5.000146e+07  5.000154e+07  5.000154e+07  5.000129e+07   
1227  5.000149e+07  5.000149e+07  5.000157e+07  5.000157e+07  5.000132e+07   
1228  5.000148e+07  5.000148e+07  5.000156e+07  5.000156e+07  5.000132e+07   
1229  5.000148e+07  5.000148e+07  5.000155e+07  5.000155e+07  5.000131e+07   
2500  1.453439e+03  1.453439e+03  1.532588e+03  1.532588e+03  1.291054e+03   

                                                
                74            75            76  
1226  5.000129e+07  5.000003e+07  5.000003e+07  
1227  5.000132e+07  5.000006e+07  5.000006e+07  
1228  5.000132e+07  5.000007e+07  5.000007e+07  
1229  5.000131e+07  5.000007e+07  5.000007e+07  
2500  1.291054e+03  2.620384e+01  2.620384e+01  

[5 rows x 81 columns]

In [34]:
origins_join2 = origins_join_rename.copy()

In [35]:
origins_join2.to_csv(os.path.join(out_folder, 'OD_11_11_RASTER.csv'))

In [36]:
origins_snapped = res_df.iloc[:,0:4]

In [37]:
origins_snapped = gpd.GeoDataFrame(origins_snapped, geometry='xy', crs='EPSG:4326')

In [38]:
origins_snapped.rename(columns={'xy':'geometry'}, inplace=True)

#### Convert to raster and save results

In [56]:
raster_path = out_pop_surface_std

In [57]:
output_path = os.path.join(out_folder, "access_ras")
if not os.path.exists(output_path):
    os.mkdir(output_path)

### Make rasters of min travel time to each dest

In [40]:
output_path = os.path.join(out_folder, "travel_time_ras")
if not os.path.exists(output_path):
    os.mkdir(output_path)

In [42]:
city_min = pd.DataFrame(origins_join2['city'].min(axis=1).apply(lambda x: (x/60)), columns=["tt_city"])
ports_min = pd.DataFrame(origins_join2['port'].min(axis=1).apply(lambda x: (x/60)), columns=["tt_port"])
airports_min = pd.DataFrame(origins_join2['airport'].min(axis=1).apply(lambda x: (x/60)), columns=["tt_airport"])
borders_min = pd.DataFrame(origins_join2['border'].min(axis=1).apply(lambda x: (x/60)), columns=["tt_border"])

In [43]:
origins_tt = origins_snapped.join([city_min, airports_min, capital_tt, ports_min, borders_min])

In [46]:
raster_path

'/home/wb514197/data/INFRA_SAP/MOZ/output/WP_2020_1km_STD.tif'

In [ ]:
aggregator.rasterize_gdf(origins_tt, 'tt_city', raster_path, os.path.join(output_path,f"cities_min_tt.tif"))
aggregator.rasterize_gdf(origins_tt, 'tt_port', raster_path, os.path.join(output_path,f"port_min_tt.tif"))
aggregator.rasterize_gdf(origins_tt, 'tt_airport', raster_path, os.path.join(output_path,f"airport_min_tt.tif"))
aggregator.rasterize_gdf(origins_tt, 'tt_border', raster_path, os.path.join(output_path,f"borders_min_tt.tif"))